# Imports and setting up viz

NB : conda env1 on PC, lam1env on spirit (Python3.12)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../../python_tools/')
from tools import *
from tools_mapping import *
from tools_hf import *
from tools_LIAISE import *

# Load files and edit datasets

LAM output analysis.

Simu irr et simu no-irr.

In [ ]:
# lon_min=-10
# lon_max=4
# lat_min=34
# lat_max=46

In [ ]:
hf_dir='../../../JZ_simu_outputs/LAM/HF_outputs'

## LMDZ

In [ ]:
def format_LMDZ_HF(filename, name, color):
    ds = xr.open_mfdataset(filename)
    ds = ds.rename({'time_counter':'time'})
    ds = ds.assign_coords(time_decimal=ds.time.dt.hour + ds.time.dt.minute / 60)
    ds.attrs['name'] = name
    ds.attrs['plot_color']=color
    ds = ds.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))

    ds['sens']=-ds['sens']
    ds['flat']=-ds['flat']

    ds['ground_level'] = ds['phis'] / 9.81
    ds['ground_level'].attrs['units'] = 'm'

    ds=add_wind_speed(ds)
    ds=add_wind_direction(ds)
    ds=add_wind_10m(ds)

    #make ovap unit g/kg
    ds['ovap'] = ds['ovap']*1000
    ds['ovap'].attrs['units'] = 'g/kg'
    #same for q2m
    ds['q2m'] = ds['q2m']*1000
    ds['q2m'].attrs['units'] = 'g/kg'
    #turn psol to hPa
    ds['psol'] = ds['psol']/100
    ds['psol'].attrs['units'] = 'hPa'

    ds = compute_grid_cell_width(ds)
    ds = add_moisture_divergence(ds)
    
    return ds

In [ ]:
#noirr
filename = '{}/noirr/sim265/*_HF_histhf.nc'.format(hf_dir)
lmdz_noirr = format_LMDZ_HF(filename, 'no_irr', 'red')
lmdz_noirr

In [ ]:
#irr
filename = '{}/irr/sim266/*_HF_histhf.nc'.format(hf_dir)
lmdz_irr = format_LMDZ_HF(filename, 'irr', 'blue')
lmdz_irr

## ORCHIDEE

In [ ]:
def format_ORC_HF(filename, name, color):
    ds = xr.open_mfdataset(filename)
    ds = ds.rename({'time_counter':'time'})
    ds = ds.assign_coords(time_decimal=ds.time.dt.hour + ds.time.dt.minute / 60)
    ds.attrs['name'] = name
    ds.attrs['plot_color']= color
    ds = ds.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))

    ds['evapot']=ds['evapot'] * 3600 * 24
    ds['evapot'].attrs['units'] = 'mm/d'

    return ds

In [ ]:
filename = '{}/noirr/sim265/*HF_sechiba_out_2.nc'.format(hf_dir)
orc_noirr = format_ORC_HF(filename, 'no_irr', 'red')
orc_noirr

In [ ]:
filename = '{}/irr/sim266/*HF_sechiba_out_2.nc'.format(hf_dir)
orc_irr = format_ORC_HF(filename, 'irr', 'blue')
orc_irr

## Obs

### Masts measurements

In [ ]:
#load all July obs in one ds
cen_mto_filename='../../../obs/LIAISE/LIAISE_LA-CENDROSA_CNRM_MTO-FLUX-30MIN_L2/LIAISE_LA-CENDROSA_CNRM_MTO-FLUX-30MIN_L2_2021-07*_V3.nc'
cen_mto = format_Cendrosa_obs(cen_mto_filename)
cen_mto

In [ ]:
sop_cen_mto = filter_xarray_by_timestamps(cen_mto, SOP_start_time, SOP_end_time)

In [ ]:
filename =  '../../../obs/LIAISE/LIAISE_ELS-PLANS_UKMO_MTO-30MIN_L2/LIAISE_ELS-PLANS_UKMO_MTO-30MIN_L2_20210721_V4.0.dat'
ds = dat_to_xarray(filename, 152, 153)
els_mto_2107 = format_ElsPlans_obs(ds, start_day='2021-07-21')

filename =  '../../../obs/LIAISE/LIAISE_ELS-PLANS_UKMO_MTO-30MIN_L2/LIAISE_ELS-PLANS_UKMO_MTO-30MIN_L2_20210722_V4.0.dat'
ds = dat_to_xarray(filename, 152, 153)
els_mto_2207 = format_ElsPlans_obs(ds, start_day='2021-07-22')

In [ ]:
july_days=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
for day in july_days:
    filename =  '../../../obs/LIAISE/LIAISE_ELS-PLANS_UKMO_MTO-30MIN_L2/LIAISE_ELS-PLANS_UKMO_MTO-30MIN_L2_202107{}_V4.0.dat'.format(day)
    ds = dat_to_xarray(filename, 152, 153)
    day_els_mto = format_ElsPlans_obs(ds, start_day='2021-07-{}'.format(day))
    if day=='01':
        els_mto=day_els_mto
    else:
        els_mto=xr.concat([els_mto,day_els_mto],dim='time')
els_mto

In [ ]:
sop_els_mto = filter_xarray_by_timestamps(els_mto, SOP_start_time, SOP_end_time)

## Masking and selecting subdatasets

### IP continental subdomain

In [ ]:
# Define masks
 
con_mask_2D_ORC= orc_noirr.mean(dim='time')['Contfrac']>0.95
con_mask_2D_ORC.attrs['name']='Continental mask (95%)'

ip_polygon_mask_2D=polygon_to_mask(orc_noirr.mean(dim="time"), iberian_peninsula)
ip_polygon_mask_2D.attrs['name']='Iberian Peninsula polygon mask'

#combine two masks
ip_mask = con_mask_2D_ORC * ip_polygon_mask_2D
ip_mask.attrs['name']='IPmask'

In [ ]:
#ORC
ip_orc_noirr = apply_2Dmask_to_dataset(orc_noirr, ip_mask, dsname='no_irr')
ip_orc_irr = apply_2Dmask_to_dataset(orc_irr, ip_mask, dsname='irr')

In [ ]:
#LMDZ
ip_lmdz_noirr = apply_2Dmask_to_dataset(lmdz_noirr, ip_mask, dsname='no_irr')
ip_lmdz_irr = apply_2Dmask_to_dataset(lmdz_irr, ip_mask, dsname='irr')

### LIAISE sites

In [ ]:
#one grid cell
#highirr
Cendrosa_lon=0.6
Cendrosa_lat=41.7
#lowirr
ElsPlans_lon=1.2
ElsPlans_lat=41.46

In [ ]:
cen_lmdz_noirr = lmdz_noirr.sel(lon=Cendrosa_lon,lat=Cendrosa_lat,method='nearest').copy()
cen_lmdz_noirr.attrs['name']='La Cendrosa no_irr'
cen_lmdz_irr = lmdz_irr.sel(lon=Cendrosa_lon,lat=Cendrosa_lat,method='nearest').copy()
cen_lmdz_irr.attrs['name']='La Cendrosa irr'
cen_orc_noirr = orc_noirr.sel(lon=Cendrosa_lon,lat=Cendrosa_lat,method='nearest').copy()
cen_orc_noirr.attrs['name']='La Cendrosa no_irr'
cen_orc_irr = orc_irr.sel(lon=Cendrosa_lon,lat=Cendrosa_lat,method='nearest').copy()
cen_orc_irr.attrs['name']='La Cendrosa irr'

In [ ]:
els_lmdz_noirr = lmdz_noirr.sel(lon=ElsPlans_lon,lat=ElsPlans_lat,method='nearest').copy()
els_lmdz_noirr.attrs['name']='Els Plans no_irr'
els_lmdz_irr = lmdz_irr.sel(lon=ElsPlans_lon,lat=ElsPlans_lat,method='nearest').copy()
els_lmdz_irr.attrs['name']='Els Plans irr'
els_orc_noirr = orc_noirr.sel(lon=ElsPlans_lon,lat=ElsPlans_lat,method='nearest').copy()
els_orc_noirr.attrs['name']='Els Plans no_irr'
els_orc_irr = orc_irr.sel(lon=ElsPlans_lon,lat=ElsPlans_lat,method='nearest').copy()
els_orc_irr.attrs['name']='Els Plans irr'

In [ ]:
#add attr linestyle -- to els_ datasets
els_lmdz_noirr.attrs['linestyle']='--'
els_lmdz_irr.attrs['linestyle']='--'
els_orc_noirr.attrs['linestyle']='--'
els_orc_irr.attrs['linestyle']='--'
#add attr linestyle - to cen_ datasets
cen_lmdz_noirr.attrs['linestyle']='-'
cen_lmdz_irr.attrs['linestyle']='-'
cen_orc_noirr.attrs['linestyle']='-'
cen_orc_irr.attrs['linestyle']='-'

In [ ]:
cendrosa_gl= cen_lmdz_noirr['ground_level'].mean(dim='time').values.item()
elsplans_gl= els_lmdz_noirr['ground_level'].mean(dim='time').values.item()
cendrosa_gl,  elsplans_gl

### Golden days

In [ ]:
start_time = '2021-07-17T00:30:00.000000000'
day1_end_time = '2021-07-17T23:30:00.000000000'
day2_start_time = '2021-07-20T00:30:00.000000000'
end_time='2021-07-20T23:30:00.000000000'

In [ ]:
day1_lmdz_noirr = filter_xarray_by_timestamps(ip_lmdz_noirr, start_time, day1_end_time)
day1_lmdz_irr = filter_xarray_by_timestamps(ip_lmdz_irr, start_time, day1_end_time)
day1_orc_noirr = filter_xarray_by_timestamps(ip_orc_noirr, start_time, day1_end_time)
day1_orc_irr = filter_xarray_by_timestamps(ip_orc_irr, start_time, day1_end_time)

day2_lmdz_noirr = filter_xarray_by_timestamps(ip_lmdz_noirr, day2_start_time, end_time)
day2_lmdz_irr = filter_xarray_by_timestamps(ip_lmdz_irr, day2_start_time, end_time)     
day2_orc_noirr = filter_xarray_by_timestamps(ip_orc_noirr, day2_start_time, end_time)
day2_orc_irr = filter_xarray_by_timestamps(ip_orc_irr, day2_start_time, end_time)

In [ ]:
gold_cen_lmdz_noirr = filter_xarray_by_timestamps(cen_lmdz_noirr, start_time, end_time)
gold_cen_lmdz_irr = filter_xarray_by_timestamps(cen_lmdz_irr, start_time, end_time)
gold_cen_orc_noirr = filter_xarray_by_timestamps(cen_orc_noirr, start_time, end_time)
gold_cen_orc_irr = filter_xarray_by_timestamps(cen_orc_irr, start_time, end_time)

gold_els_lmdz_noirr = filter_xarray_by_timestamps(els_lmdz_noirr, start_time, end_time)
gold_els_lmdz_irr = filter_xarray_by_timestamps(els_lmdz_irr, start_time, end_time)
gold_els_orc_noirr = filter_xarray_by_timestamps(els_orc_noirr, start_time, end_time)
gold_els_orc_irr = filter_xarray_by_timestamps(els_orc_irr, start_time, end_time)

In [ ]:
#day1
day1_cen_lmdz_noirr = filter_xarray_by_timestamps(cen_lmdz_noirr, start_time, day1_end_time)
day1_cen_lmdz_irr = filter_xarray_by_timestamps(cen_lmdz_irr, start_time, day1_end_time)
day1_cen_orc_noirr = filter_xarray_by_timestamps(cen_orc_noirr, start_time, day1_end_time)
day1_cen_orc_irr = filter_xarray_by_timestamps(cen_orc_irr, start_time, day1_end_time)

day1_els_lmdz_noirr = filter_xarray_by_timestamps(els_lmdz_noirr, start_time, day1_end_time)
day1_els_lmdz_irr = filter_xarray_by_timestamps(els_lmdz_irr, start_time, day1_end_time)
day1_els_orc_noirr = filter_xarray_by_timestamps(els_orc_noirr, start_time, day1_end_time)
day1_els_orc_irr = filter_xarray_by_timestamps(els_orc_irr, start_time, day1_end_time)

#day2
day2_cen_lmdz_noirr = filter_xarray_by_timestamps(cen_lmdz_noirr, day2_start_time, end_time)
day2_cen_lmdz_irr = filter_xarray_by_timestamps(cen_lmdz_irr, day2_start_time, end_time)
day2_cen_orc_noirr = filter_xarray_by_timestamps(cen_orc_noirr, day2_start_time, end_time)
day2_cen_orc_irr = filter_xarray_by_timestamps(cen_orc_irr, day2_start_time, end_time)

day2_els_lmdz_noirr = filter_xarray_by_timestamps(els_lmdz_noirr, day2_start_time, end_time)
day2_els_lmdz_irr = filter_xarray_by_timestamps(els_lmdz_irr, day2_start_time, end_time)
day2_els_orc_noirr = filter_xarray_by_timestamps(els_orc_noirr, day2_start_time, end_time)
day2_els_orc_irr = filter_xarray_by_timestamps(els_orc_irr, day2_start_time, end_time)

# Maps

In [ ]:
var='moisture_divergence'
ds=ip_lmdz_irr

vmin=0
vmax=1000
# vmin,vmax = None, None
cmap=reds
# title='off'
title=None
# title='Evaporative fraction (irr, JJA)'
# clabel='Irrigation (mm/d)'
clabel=None
map_ave(ds, var, vmin=vmin, vmax=vmax, cmap=cmap,
        title=title, clabel=clabel)

In [ ]:
#Diff map
var='moisture_convergence'
ds1=ip_lmdz_irr
ds2=ip_lmdz_noirr

vmax= 1.5
vmin=-1.5
# vmin,vmax = None, None
cmap=emb_neutral
title=None
map_diff_ave(ds1, ds2, var, vmin=vmin, vmax=vmax, cmap=cmap, 
             sig=False,
             title=title, clabel=clabel)

In [ ]:
Cendrosa_Latitude=41.69336
Cendrosa_Longitude=0.928538
Cenrosa_Altitude= 240

ElsPlans_Latitude = 41.590111
ElsPlans_Longitude = 1.029363
ElsPlans_Altitude = 334

In [ ]:
#Display map of var for ds on restricted area
var='irrigation'
ds = ip_orc_irr
# ds = lmdz_irr

latmin=41.35
latmax=41.9
lonmin=0.6
lonmax=1.4
# latmin=40.5
# latmax=43.5
# lonmin=-3
# lonmax=3
cmap=myterrain
cmap=wet
vmin=0
vmax=1000
vmax=2
# vmin, vmax=None, None

title='Irrigation (July 2021, mm/d)'
# title=None

#restrict ds to latmin, latmax, lonmin, lonmax
ds = ds.where(ds.lat >= latmin, drop=True).where(ds.lat <= latmax, drop=True)
ds = ds.where(ds.lon >= lonmin, drop=True).where(ds.lon <= lonmax, drop=True)
plotvar = ds[var].mean(dim='time')

map_plotvar(plotvar, cmap=cmap, vmin=vmin,vmax=vmax, title=title)

plt.plot(Cendrosa_lon, Cendrosa_lat, 'go', markersize=10)
plt.plot(ElsPlans_lon, ElsPlans_lat, 'go', markersize=10)
plt.plot(Cendrosa_Longitude, Cendrosa_Latitude, 'ro', markersize=10)
plt.plot(ElsPlans_Longitude, ElsPlans_Latitude, 'ro', markersize=10)

In [ ]:
#map winds
ds = ip_lmdz_irr

#restrict ds to latmin, latmax, lonmin, lonmax
latmin=40.5
latmax=43.5
lonmin=-3
lonmax=3
ds = ds.where(ds.lat >= latmin, drop=True).where(ds.lat <= latmax, drop=True)
ds = ds.where(ds.lon >= lonmin, drop=True).where(ds.lon <= lonmax, drop=True)

map_wind(ds,
        #  extra_var='moisture_convergence', extra_ds=ds,
         height='850', dist=2, scale=30,
         cmap=reds,
         vmin=-0,
         vmax= 5, 
        #  clabel=None
         )

#add liaise points
# plt.plot(Cendrosa_lon, Cendrosa_lat, 'go', markersize=10)
# plt.plot(ElsPlans_lon, ElsPlans_lat, 'go', markersize=10)
plt.plot(Cendrosa_Longitude, Cendrosa_Latitude, 'bo', markersize=10)
plt.plot(ElsPlans_Longitude, ElsPlans_Latitude, 'bo', markersize=10)

In [ ]:
ds1 = day2_lmdz_irr
ds2 = day2_lmdz_noirr

#restrict ds to latmin, latmax, lonmin, lonmax
latmin=40.5
latmax=43.5
lonmin=-3
lonmax=3
ds1 = ds1.where(ds1.lat >= latmin, drop=True).where(ds1.lat <= latmax, drop=True)
ds1 = ds1.where(ds1.lon >= lonmin, drop=True).where(ds1.lon <= lonmax, drop=True)
ds2 = ds2.where(ds2.lat >= latmin, drop=True).where(ds2.lat <= latmax, drop=True)
ds2 = ds2.where(ds2.lon >= lonmin, drop=True).where(ds2.lon <= lonmax, drop=True)

map_wind_diff(ds1, ds2,
         height='10m', dist=1, scale=2,
         cmap=emb_neutral,
         vmin=-0.2,
         vmax= 0.2, 
        #  clabel=None
         )

#add liaise points
# plt.plot(Cendrosa_lon, Cendrosa_lat, 'go', markersize=10)
# plt.plot(ElsPlans_lon, ElsPlans_lat, 'go', markersize=10)
plt.plot(Cendrosa_Longitude, Cendrosa_Latitude, 'bo', markersize=10)
plt.plot(ElsPlans_Longitude, ElsPlans_Latitude, 'bo', markersize=10)

In [ ]:
ds = day2_lmdz_irr

#restrict ds to latmin, latmax, lonmin, lonmax
latmin=40.5
latmax=43.5
lonmin=-3
lonmax=3
ds = ds.where(ds.lat >= latmin, drop=True).where(ds.lat <= latmax, drop=True)
ds = ds.where(ds.lon >= lonmin, drop=True).where(ds.lon <= lonmax, drop=True)

map_moisture_transport(ds,
        #  extra_var='moisture_convergence'
         dist=2, scale=1000,
         cmap=blues,
         vmin=-0,
         vmax= 250, 
        #  clabel=None
         )

#add liaise points
# plt.plot(Cendrosa_lon, Cendrosa_lat, 'go', markersize=10)
# plt.plot(ElsPlans_lon, ElsPlans_lat, 'go', markersize=10)
plt.plot(Cendrosa_Longitude, Cendrosa_Latitude, 'bo', markersize=10)
plt.plot(ElsPlans_Longitude, ElsPlans_Latitude, 'bo', markersize=10)


In [ ]:
ds1 = day2_lmdz_irr
ds2 = day2_lmdz_noirr

#restrict ds to latmin, latmax, lonmin, lonmax
latmin=40.5
latmax=43.5
lonmin=-3
lonmax=3
ds1 = ds1.where(ds1.lat >= latmin, drop=True).where(ds1.lat <= latmax, drop=True)
ds1 = ds1.where(ds1.lon >= lonmin, drop=True).where(ds1.lon <= lonmax, drop=True)
ds2 = ds2.where(ds2.lat >= latmin, drop=True).where(ds2.lat <= latmax, drop=True)
ds2 = ds2.where(ds2.lon >= lonmin, drop=True).where(ds2.lon <= lonmax, drop=True)

map_moisture_transport_diff(ds1, ds2,
         dist=2, scale=20,
         cmap=emb_neutral,
         vmin=-3,
         vmax= 3, 
        #  clabel=None
         )

#add liaise points
# plt.plot(Cendrosa_lon, Cendrosa_lat, 'go', markersize=10)
# plt.plot(ElsPlans_lon, ElsPlans_lat, 'go', markersize=10)
plt.plot(Cendrosa_Longitude, Cendrosa_Latitude, 'bo', markersize=10)
plt.plot(ElsPlans_Longitude, ElsPlans_Latitude, 'bo', markersize=10)

# Time series and diurnal cycle

In [ ]:
var='precip'

# ds1=gold_cen_lmdz_irr
# ds2=gold_cen_lmdz_noirr
ds1=gold_els_lmdz_irr
ds2=gold_els_lmdz_noirr

# ds1=gold_cen_orc_irr
# ds2=gold_cen_orc_noirr
# ds1=gold_els_orc_irr
# ds2=gold_els_orc_noirr

ds3=gold_cen_mto
ds3=gold_els_mto

ds_list=[ds1, ds2, ds3]#, ds4]
ds_list=[ds3]

vmin=-0
vmax=500
vmin, vmax=None, None
title='{} ({})'.format(var, ds1[var].attrs['units'])
# title=None

time_series_ave(ds_list, var, ds_colors=True, ds_linestyle=True, title=title, vmin=vmin, vmax=vmax)

In [ ]:
var='s_pblh'

ds1=cen_lmdz_noirr
ds2=cen_lmdz_irr
# ds1=ip_orc_noirr
# ds2=ip_orc_irr
ds_list=[ds1, ds2]#, ds3]
# ds_list=[ds2]

title=None
diurnal_cycle_ave(ds_list, var, title='{} ({})'.format(var, ds1[var].attrs['units']))

# Energy budget

In [ ]:
## FOR ONE SITE, DISTINGUISHING BY SIM
ds_lmdz_irr = cen_lmdz_irr
ds_orc_irr  = cen_orc_irr
ds_lmdz_noirr = cen_lmdz_noirr
ds_orc_noirr  = cen_orc_noirr
title='La Cendrosa'
energy_budget_dc(ds_lmdz_irr, ds_lmdz_noirr, ds_orc_irr, ds_orc_noirr, title=title, lab1='irr', lab2='noirr')

ds_lmdz_irr = els_lmdz_irr
ds_orc_irr  = els_orc_irr
ds_lmdz_noirr = els_lmdz_noirr
ds_orc_noirr  = els_orc_noirr
title='Els Plans'
energy_budget_dc(ds_lmdz_irr, ds_lmdz_noirr, ds_orc_irr, ds_orc_noirr, title=title, lab1='irr', lab2='noirr')

In [ ]:
## FOR ONE SIM, DISTINGUISHING BY SITE
ds_lmdz1 = cen_lmdz_irr
ds_orc1 = cen_orc_irr
ds_lmdz2 = els_lmdz_irr
ds_orc2 = els_orc_irr
title = 'Irrigated sim'
energy_budget_dc(ds_lmdz1, ds_lmdz2, ds_orc1, ds_orc2, title=title, lab1='La Cendrosa', lab2='Els Plans')

ds_lmdz1 = cen_lmdz_noirr
ds_orc1 = cen_orc_noirr
ds_lmdz2 = els_lmdz_noirr
ds_orc2 = els_orc_noirr
title = 'Non-irrigated sim'
energy_budget_dc(ds_lmdz1, ds_lmdz2, ds_orc1, ds_orc2, title=title, lab1='La Cendrosa', lab2='Els Plans')

# Vertical profiles

## Sims only

In [ ]:
var='theta'
ds1=els_lmdz_irr
ds2=els_lmdz_noirr
ds3=cen_lmdz_irr
ds4=cen_lmdz_noirr

# ds_list=[ds1, ds2, ds3, ds4]
ds_list=[ds3, ds4]
ds_list=[ds1]
ds_list=[gold_cen_lmdz_irr]

obs_ds_list=[Cendrosa_RS]
obs_ds_list=None

altmax=3500

title=None

# profile_preslevs_local(ds_list, var, title=title, preslevelmax=preslevelmax)
profile_altitude_local_mean(ds_list, var, title=title, altmax=altmax,
                            obs_ds_list=obs_ds_list,
                            nbins=None,
                            substract_gl=True,
                            )

In [ ]:
var='theta'
ds1=day1_els_lmdz_irr
ds2=day1_els_lmdz_noirr
ds3=day1_cen_lmdz_irr
ds4=day1_cen_lmdz_noirr

ds_list=[ds1, ds2, ds3, ds4]
# ds_list=[ds3,ds4]
# ds_list=[ds1]

altmin=-0
altmax=3500

# times=[1.5, 4.5, 7.5, 10.5, 13.5, 16.5, 19.5, 22.5]
times=[4.5, 7.5, 10.5, 13.5, 16.5, 19.5]
# times=[0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5, 10.5, 11.5, 12.5, 13.5, 14.5, 15.5, 16.5, 17.5, 18.5, 19.5, 20.5, 21.5, 22.5, 23.5]
# times=[5.5, 7.5, 9.5, 11.5, 13.5, 15.5, 17.5, 19.5, 21.5]
# times=[12.5]

xmin=298
xmax=312
# xmin, xmax=None, None

profile_altitude_multipletimes_mean(ds_list, var, times, altmin=altmin, altmax=altmax,
                                    xmin=xmin, xmax=xmax,
                                    substract_gl=True)

In [ ]:
var='theta'
ds1=day1_els_lmdz_irr
ds2=day1_els_lmdz_noirr
ds3=day1_cen_lmdz_irr
ds4=day1_cen_lmdz_noirr

ds=ds4

altmin=-0
altmax=3500

times=[1.5, 4.5, 7.5, 10.5, 13.5, 16.5, 19.5, 22.5]
times=[4.5, 7.5, 10.5, 13.5, 16.5, 19.5]
times=[0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5, 10.5, 11.5, 12.5, 13.5, 14.5, 15.5, 16.5, 17.5, 18.5, 19.5, 20.5, 21.5, 22.5, 23.5]
# times=[5.5, 7.5, 9.5, 11.5, 13.5, 
times=[11.5, 12.5, 13.5, 14.5, 15.5, 16.5, 17.5]
# times=[12.5]

xmin=308
xmax=318
# xmin, xmax=None, None

profile_altitude_multipletimes_mean_singleplot(ds, var, times, altmin=altmin, altmax=altmax,
                                                  xmin=xmin, xmax=xmax,
                                                  substract_gl=True)

## With obs

In [ ]:
#obs alone
var='theta'
ds1=cen_RS_d2[12]

ds_list=[ds1]

altmax=2500

title=ds1.attrs['name']

profile_altitude_obs(ds_list, var, title=title, altmax=altmax,
                        xmin=306,
                        xmax=315,
                       nbins=None,
                       substract_gl=True,
                       )

In [ ]:
ds1=els_lmdz_irr
ds2=els_lmdz_noirr
ds3=cen_lmdz_irr
ds4=cen_lmdz_noirr

timestamp='2021-07-22T12:01:00.000000000'

ds_list=[ds1, ds2]#, ds3, ds4]
# ds_list=[ds3, ds4]
# ds_list=[ds3]

obs_ds_list=[Cendrosa_RS]
obs_ds_list=[ElsPlans_RS]
# obs_ds_list=None

altmax=2500
title=None

var='theta'
var, xmin, xmax = 'theta', 306, 314
var, xmin, xmax = 'ovap', 2, 14
var, xmin, xmax = 'wind_speed', 0, 9
var, xmin, xmax = 'wind_direction', 0, 360
# xmin, xmax = None, None

# profile_preslevs_local(ds_list, var, title=title, preslevelmax=preslevelmax)
profile_altitude_local_timestamp(ds_list, var, timestamp, title=title, altmax=altmax, xmin=xmin, xmax=xmax, 
                            obs_ds_list=obs_ds_list,
                            nbins=5,
                            substract_gl=True,
                            )

In [ ]:
ds1=day1_els_lmdz_irr
ds2=day1_els_lmdz_noirr
ds3=cen_lmdz_irr
ds4=cen_lmdz_noirr

ds_list, obs_dict=[day1_els_lmdz_irr, day1_els_lmdz_noirr], els_RS_d1
ds_list, obs_dict=[day1_cen_lmdz_irr, day1_cen_lmdz_noirr], cen_RS_d1

# ds_list, obs_dict=[day2_els_lmdz_irr, day2_els_lmdz_noirr], els_RS_d2
# ds_list, obs_dict=[day2_cen_lmdz_irr, day2_cen_lmdz_noirr], cen_RS_d2
# ds_list=[ds3]

altmax=2500
title=None

times=[5.5, 7.5, 9.5, 11.5, 13.5, 15.5, 17.5]
# times=els_d1_times+0.5
# times=[x + 0.5 for x in cen_d1_times]
times=[12.5]

var='theta'
var, xmin, xmax = 'theta', 298, 314
# var, xmin, xmax = 'ovap', 2, 14
# var, xmin, xmax = 'wind_speed', 0, 9
# var, xmin, xmax = 'wind_direction', 0, 360
# xmin, xmax = None, None

# profile_preslevs_local(ds_list, var, title=title, preslevelmax=preslevelmax)
profile_altitude_multipletimes_obs(ds_list, obs_dict, var, times, 
                                 # title=title, 
                                 altmax=altmax, xmin=xmin, xmax=xmax, 
                                 # nbins=5,
                                 substract_gl=True,
                                )